In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

# Figure 4

In [ ]:
temp_models = {}
qvals = [0.1, 0.25, 0.5, 0.75, 0.9]
qvalsmean = [f"{q}-mean" for q in qvals]
# qvalsmax = [f"{q}-max" for q in qvals]

# for q in qvalsmean + qvalsmax + ["mean_zone_temp"]:
for q in qvalsmean:
    with_plots = q == "0.5-mean"
    temp_models[q] = {}
    if with_plots:
        f, ax = plt.subplots(
            2, 3, figsize=(pu.PAGE_WIDTH, 2 * pu.ROW_HEIGHT), sharey=True, sharex=True
        )
        ax = ax.flatten()
    for i, project in enumerate(pu.PROJECTS):
        weekend = pu.choose_weekend[project]
        if (project == "OFF-4") and (q == "mean_zone_temp"):
            continue  # Issue with this one
        if q == "mean_zone_temp":
            ylabel = f"Mean zone temperature ($\degree$F)"
        else:
            ylabel = f"Zone temperatures ($\degree$F)\n{q.split('-')[1]} {float(q.split('-')[0])*100:.0f}\u1D57\u02B0 percentile"
        if with_plots:
            pu.plot_base(
                data[project], y=q, ylabel=ylabel, fax=(f, ax[i]), weekend=weekend
            )
            fax = (f, ax[i])
        else:
            fax = None
        temp_models[q][project] = pu.create_model(
            data[project], fax=fax, weekend=weekend, y=q, log=False
        )
        if with_plots:
            ax[i].set_title(f"{pu.plot_numbers[i]}) {project}")

    if with_plots:
        for a in ax:
            a.yaxis.set_tick_params(labelleft=True)
            a.xaxis.set_tick_params(labelbottom=True)
        f.tight_layout()
        pu.add_legend((f, ax))
        if pu.save_fig:
            f.savefig(pu.fig_path / "Figure 4 A-F.pdf")

In [ ]:
color = [pu.grey1, pu.grey2, pu.dark_grey, pu.grey2, pu.grey1]
ms = [1, 1, 2, 1, 1]

f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 0.75))
ax = ax.flatten()

j = -0.2
for iq, q in enumerate(qvalsmean):
    for i, project in enumerate(pu.PROJECTS):
        pu.ci_plot(
            ax[0],
            i + j,
            temp_models[q][project],
            "Treated",
            color=color[iq],
            ms=ms[iq],
            log=False,
        )
        pu.ci_plot(
            ax[1],
            i + j,
            temp_models[q][project],
            "Tmean",
            color=color[iq],
            ms=ms[iq],
            log=False,
        )
    j += 0.1


for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=30, ha="right")
ax[0].set_ylabel("Impact (F)")
ax[1].set_ylabel("Impact (F/F)")

title0 = (
    f"{pu.plot_numbers[6]}) "
    + "Impact of 2$\degree$F set point increase on zone temp. percentiles\n(10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)"
)
title1 = (
    f"{pu.plot_numbers[7]}) "
    + "Impact of mean daily OAT on zone temp. percentiles\n(10\u1D57\u02B0, 25\u1D57\u02B0, 50\u1D57\u02B0, 75\u1D57\u02B0, 90\u1D57\u02B0)"
)

ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.8, bottom=0.3)

if pu.save_fig:
    f.savefig(pu.fig_path / "Figure 4 G-H.pdf")

In [ ]:
def y_transform(y):
    return 10 * y


print("Tmean impact for 10F")
txt = ""
for q in temp_models:
    txt += f'\t{q.split("-")[0]}'
print(txt)
for project in pu.PROJECTS:
    txt = f"{project}:"
    for q in temp_models:
        mod = temp_models[q][project]
        txt += f'\t{y_transform(mod.params["Tmean"]):.2f}'
    print(txt)

# Table S5

In [ ]:
with open(pu.si_path / "Table S5.tex", "w") as fw:
    fw.write(
        pu.si_table(
            temp_models["0.5-mean"],
            pu.PROJECTS,
            caption="Additional numerical data for models shown in Figure 4 (room temperatures).",
            label="table:s5",
        )
    )